In [63]:
import pandas as pd

# Cleaning a .txt dataset using pandas

Some datasets may come in the form of a list. In this example, we will use [this dataset](https://raw.githubusercontent.com/realpython/python-data-cleaning/master/Datasets/university_towns.txt) which contains a list of universities by state. 

Pandas has a ```.read_fwf()```method that allows you to read a ```.txt``` file and parse it as a dataframe. We will set ```header = None``` to avoid setting the first element of the list as a column name.

After reading the datasets with pandas, we print the first rows to see what the different elements look like.

In [41]:
town = pd.read_fwf('https://raw.githubusercontent.com/realpython/python-data-cleaning/master/Datasets/university_towns.txt', header = None)
town

,0
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]
...,...
562,Stevens Point (University of Wisconsin–Stevens...
563,Waukesha (Carroll University)
564,Whitewater (University of Wisconsin–Whitewater...
565,Wyoming[edit]


After printing some rows from the dataframe, we notice that the name of each state is followed by ```'[edit]'```, and the rows following the name of the state are either universities or colleges located in that state. Another thing that stands out is that the name of the city comes right before the university/college name.

In summary, the structure of the dataframe looks something like this:

```
State[edit]
City (University)
City (University)
City (University)
State[edit]
City (University)
City (University)
...
...
...
```

Ideally, we would want a dataframe with three columns: ```State```, ```RegionName```, and ```UniversityName```.


### Step 1: Create a tuple with state and university names
We iterate through each element (```line```) in the column, identify the rows containing ```'[edit]'```, and  replace it with and empty string ```''```. We want to temporarily store ```line``` as ```state```. All ```line``` elements that come before the next element containing ```'[edit]'``` will be stored in a tuple containing the name of the state and the region and university name: (```state, line```). Finally, we append the tuple to the empty list ```university_towns```.


In [42]:
university_towns = []
for line in town.iloc[:,0]:
    if '[edit]' in line:
        # Remember this 'state' until the next is found
        line.replace('[edit]', '')
        state = line
    else:
        # Otherwise, we have a city; keep 'state' as last-seen
        university_towns.append((state, line))

In [43]:
university_towns[:5]

[('Alabama[edit]', 'Auburn (Auburn University)[1]'),
 ('Alabama[edit]', 'Florence (University of North Alabama)'),
 ('Alabama[edit]', 'Jacksonville (Jacksonville State University)[2]'),
 ('Alabama[edit]', 'Livingston (University of West Alabama)[2]'),
 ('Alabama[edit]', 'Montevallo (University of Montevallo)[2]')]

We turn the tuples in ```university_towns``` into a dataframe.


In [44]:
towns_df = pd.DataFrame(university_towns, columns=['State', 'RegionName'])
towns_df.head()

,State,RegionName
0,Alabama[edit],Auburn (Auburn University)[1]
1,Alabama[edit],Florence (University of North Alabama)
2,Alabama[edit],Jacksonville (Jacksonville State University)[2]
3,Alabama[edit],Livingston (University of West Alabama)[2]
4,Alabama[edit],Montevallo (University of Montevallo)[2]


### Step 2: Remove `[edit]` from state name

We use a lambda function that removes ```'[edit]'``` from the name of the states.

In [45]:
towns_df['State'] = towns_df['State'].map(lambda x: x.rstrip('[edit]'))
towns_df

,State,RegionName
0,Alabama,Auburn (Auburn University)[1]
1,Alabama,Florence (University of North Alabama)
2,Alabama,Jacksonville (Jacksonville State University)[2]
3,Alabama,Livingston (University of West Alabama)[2]
4,Alabama,Montevallo (University of Montevallo)[2]
...,...,...
512,Wisconsin,River Falls (University of Wisconsin–River Fal...
513,Wisconsin,Stevens Point (University of Wisconsin–Stevens...
514,Wisconsin,Waukesha (Carroll University)
515,Wisconsin,Whitewater (University of Wisconsin–Whitewater...


### Step 3: Extract the name of university from `RegionName` and create a new column `UniversityName`.

The ```RegionName``` column contains both the name of the university/college and city. In order to create a new column ```UniversityName``` we extract all strings in between parenthesis and append it to an empty list ```university``` that will become the new column with the university names.

In [46]:
university = []
for i in towns_df['RegionName']:
    if '(' in i:
        uni = i.split(' (')[1]
    else: 
        uni = i

    if ')' in uni:
        uni = uni.split(')')[0]
  
    university.append(uni)

towns_df['UniversityName'] = university

In [47]:
towns_df.head(5)

,State,RegionName,UniversityName
0,Alabama,Auburn (Auburn University)[1],Auburn University
1,Alabama,Florence (University of North Alabama),University of North Alabama
2,Alabama,Jacksonville (Jacksonville State University)[2],Jacksonville State University
3,Alabama,Livingston (University of West Alabama)[2],University of West Alabama
4,Alabama,Montevallo (University of Montevallo)[2],University of Montevallo


### Step 4: Clean up `RegionName`

Following a similar logic from step 3, we split each element in ```RegionName``` on ```'('``` and keep the very first part of the string. 

In [48]:
regions = []

for i in towns_df['RegionName']:
    if '(' in i:
        region = i.split(' (')[0]
    else: 
        region = i
  
    regions.append(region)

towns_df['RegionName'] = regions

In [49]:
towns_df

,State,RegionName,UniversityName
0,Alabama,Auburn,Auburn University
1,Alabama,Florence,University of North Alabama
2,Alabama,Jacksonville,Jacksonville State University
3,Alabama,Livingston,University of West Alabama
4,Alabama,Montevallo,University of Montevallo
...,...,...,...
512,Wisconsin,River Falls,University of Wisconsin–River Falls
513,Wisconsin,Stevens Point,University of Wisconsin–Stevens Point
514,Wisconsin,Waukesha,Carroll University
515,Wisconsin,Whitewater,University of Wisconsin–Whitewater


After taking a closer look at the university names, some of the rows contain more than one university. For example, row #6 in ```UniversityName``` contains all these names: ```Claremont McKenna College, Pomona College, Harvey Mudd College, Scripps College, Pitzer College, Keck Graduate Institute, Claremont Graduate University```. See full list of universities below.

In [50]:
for i in towns_df['UniversityName']:
    if ',' in i:
        print(i)

University of Alabama, Stillman College, Shelton State
Henderson State University, Ouachita Baptist University
Central Baptist College, Hendrix College, University of Central Arkansas
University of California, Berkeley
California State University, Chico
Claremont McKenna College, Pomona College, Harvey Mudd College, Scripps College, Pitzer College, Keck Graduate Institute, Claremont Graduate University
California State University, Sonoma
University of California, Davis
University of California, Irvine
University of California, Santa Barbara
University of California, Merced
Cal Poly Pomona, WesternU
University of California, Riverside, California Baptist University, La Sierra University
California State University, Sacramento
California State University, San Bernardino, American Sports University
University of California, San Diego, San Diego State University
Fielding Graduate University, Santa Barbara City College, University of California, Santa Barbara, Westmont College
University of

### Step 5: Split rows with multiple university names

We split each element in ```UniversityName``` on ```'('```, turn it into a list, and stack the dataframe with new indexes ```RegionName``` and ```State```. Then we reset the index and assign new column names to the new dataframe.

In [51]:
b = pd.DataFrame(towns_df.UniversityName.str.split(', ').tolist(), index=[towns_df.RegionName, towns_df.State]).stack()
b = b.reset_index()[[0, 'State', 'RegionName']] # var1 variable is currently labeled 0
b.columns = ['UniversityName', 'State', 'RegionName'] # renaming var1

In [52]:
b

,UniversityName,State,RegionName
0,Auburn University,Alabama,Auburn
1,University of North Alabama,Alabama,Florence
2,Jacksonville State University,Alabama,Jacksonville
3,University of West Alabama,Alabama,Livingston
4,University of Montevallo,Alabama,Montevallo
...,...,...,...
710,University of Wisconsin–River Falls,Wisconsin,River Falls
711,University of Wisconsin–Stevens Point,Wisconsin,Stevens Point
712,Carroll University,Wisconsin,Waukesha
713,University of Wisconsin–Whitewater,Wisconsin,Whitewater


### Step 6: Clean up ```UniversityName```

Some university names, like ```University of California, Los Angeles```, were splitted on ```','``` and, part of the university name, moved to a different row. Below, we solve this issue by comparing each element of ```RegionName``` with ```UniversityName``` and solve this issue.

Next, we take a last and closer look to the name of the universities and spot some inconsistencies. Some university names do not make sense and some other are missing the word "university" or "college". We create lists with these particular examples and iterate through each element in ```UniversityName``` to fix these names.

In [54]:
for i in range(len(b['RegionName'])):
    if b.iloc[i, 0] == b.iloc[i, 2]:
        b.iloc[i-1,0] = b.iloc[i-1,0] + ', ' + b.iloc[i, 0]
        b.drop(b.index[i], axis = 0, inplace=True)

In [56]:
l = ['Los Angeles', 'San Bernardino', 'Santa Barbara', 'Stanislaus', 'Faribault', 'Sonoma']
names = ['Spart', 'Worces, The Five College Region of Western Massachusetts:', ', The Colleges of Worcester Consortium:']
college = ['Becker', 'Assumption']

for i in range(len(b['UniversityName'])):
    if b.iloc[i, 0] in l:
        b.iloc[i-1,0] = b.iloc[i-1,0] + ', ' + b.iloc[i, 0]
        b.drop(b.index[i], axis = 0, inplace=True)
  
    elif b.iloc[i, 0] in names:
        b.drop(b.index[i], axis = 0, inplace=True)

    elif b.iloc[i, 0] in college:
        b.iloc[i, 0] = b.iloc[i, 0] + 'College'

    else:
        pass


Below, we print the university names to check the names on last time.

In [57]:
for i in range(len(b['UniversityName'])):
    print(b.iloc[i, 0])

Auburn University
University of North Alabama
Jacksonville State University
University of West Alabama
University of Montevallo
Troy University
University of Alabama
Stillman College
Shelton State
Tuskegee University
University of Alaska Fairbanks
Northern Arizona University
Arizona State University
University of Arizona
Henderson State University
Ouachita Baptist University
Central Baptist College
Hendrix College
University of Central Arkansas
University of Arkansas
Arkansas State University
Southern Arkansas University
University of Arkansas at Monticello
Arkansas Tech University
Harding University
Pacific Union College
Humboldt State University
University of California, Berkeley
California State University, Chico
Claremont McKenna College
Pomona College
Harvey Mudd College
Scripps College
Pitzer College
Keck Graduate Institute
Claremont Graduate University
California State University, Sonoma
University of California, Davis
University of California, Irvine
University of California, S

Below there is a sample of the first 15 rows of the dataframe.

In [62]:
b.head(15)

,UniversityName,State,RegionName
0,Auburn University,Alabama,Auburn
1,University of North Alabama,Alabama,Florence
2,Jacksonville State University,Alabama,Jacksonville
3,University of West Alabama,Alabama,Livingston
4,University of Montevallo,Alabama,Montevallo
5,Troy University,Alabama,Troy
6,University of Alabama,Alabama,Tuscaloosa
7,Stillman College,Alabama,Tuscaloosa
8,Shelton State,Alabama,Tuscaloosa
9,Tuskegee University,Alabama,Tuskegee
